In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:

[[rep_mobile_citizens_id_city_partner]]

[[sessions_st_mobile]]

[[citizens_dir_mobile]]
___

In [14]:
query_text = """--sql
CREATE TABLE db1.maf_rep_mobile_total
(
    `report_date` Date,
    `MAF` UInt64,
    `stricted_MAF` UInt64,
    `freemonetization_MAF` UInt64
)
ENGINE = MergeTree()
ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [15]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.maf_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 45 MINUTE TO db1.maf_rep_mobile_total AS
    WITH t_m AS (SELECT
        report_date,
        flat_uuid,
        monetization,
        citizen_id
    FROM db1.rep_mobile_citizens_id_city_partner
    WHERE toDate(`flat_created_at`) <= report_date
        AND state = 'activated'
    ),
    t_s AS 
    (SELECT 
        report_date,
        sessions_st_mobile_ch.citizen_id AS citizen_id
    FROM db1.`sessions_st_mobile_ch` AS sessions_st_mobile
    LEFT ANY JOIN  db1.`citizens_dir_mobile_ch` AS citizens_dir_mobile
            ON citizens_dir_mobile.citizen_id = sessions_st_mobile.citizen_id 
    WHERE toDateOrNull(citizens_dir_mobile.`created_at`) <= sessions_st_mobile.report_date
        AND (toDate(`last_use`) BETWEEN toStartOfMonth(sessions_st_mobile.report_date) AND sessions_st_mobile.report_date) 
    )
    --
    SELECT
        t_m.report_date AS report_date,
        COUNT(t_m.flat_uuid) AS MAF,
        COUNT(if(monetization = 0, t_m.flat_uuid, null)) AS stricted_MAF,
        COUNT(if(monetization = 1 or monetization is null, t_m.flat_uuid, null)) AS freemonetization_MAF
    FROM t_m
    ANY JOIN t_s 
        ON t_m.citizen_id = t_s.citizen_id 
        AND t_m.report_date = t_s.report_date
    GROUP BY report_date
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [8]:
query_text = """--sql
    SELECT
        *
    FROM db1.maf_rep_mobile_total
    ORDER BY report_date desc
    limit 10
    """

ch.query_run(query_text)


report_date,MAF,stricted_MAF,freemonetization_MAF
date,i64,i64,i64
2025-07-29,251908,94420,157488
2025-07-28,254301,95033,159268
2025-07-27,241872,90246,151626
2025-07-26,233492,87279,146213
2025-07-25,253788,253788,0
2025-07-24,254934,95781,159153
2025-07-23,256162,95791,160371
2025-07-22,256298,95830,160468
2025-07-21,258005,96979,161026


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.maf_rep_mobile_total DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [9]:
query_text = """--sql
    DROP TABLE db1.maf_rep_mobile_total_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop table

In [13]:
query_text = """--sql
    DROP TABLE db1.maf_rep_mobile_total
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.maf_rep_mobile_total_mv
"""

ch.query_run(query_text)